# Milestone 2 - Implementing Transfer Learning on Art images dataset using  pre-trained models  

---  

# Objective of this notebook
* Train pre-trained alexnet, vgg19 and resnet50 models using transfer learning on given art images dataset to compare and visualize their performance on different classes.

> **Note:** The content of this notebook follows the description provided in [Milestone-2](./Milestone-2.md)

---
## 1. Prerequisites

### Importing packages & modules

You might prefer to load the required modules/packages when required. Feel free to do so if it is your preference

---  


In [ ]:
# Common modules/packages
import matplotlib.pyplot as plt
import math
import numpy as np
import pathlib
import sys, shutil, time
import warnings

# PyTorch modules/packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

from torchvision import datasets, models, transforms
from PIL import ImageFile

%matplotlib inline
warnings.filterwarnings('ignore')

### Setting up the training mode based on CUDA capability

In [ ]:
device = 'cpu'  # sets the default value
train_on_gpu = torch.cuda.is_available()  # returns True if CUDA enabled GPU is available

if train_on_gpu == True :
    print('CUDA is available!  Training on GPU ...')
    print('\n')
    print(torch.cuda.get_device_properties(0))
    device = 'cuda'    

### Define some utilities functions

In [ ]:
# Retrieves the list of files with a directory
def getFilesInDirectory(pathToDir, extension = "*.*"):
    if not isinstance(pathToDir, pathlib.PurePath):
        pathToDir = pathlib.Path(pathToDir)

    return list(pathToDir.glob(extension))

# Retrieves the list of folders with a directory
def getFoldersInDirectory(pathToDir, prefix = ""):
    if not isinstance(pathToDir, pathlib.PurePath):
        pathToDir = pathlib.Path(pathToDir)

    return sorted([fld for fld in pathToDir.iterdir() if fld.is_dir() and not fld.name.lower().startswith(prefix)])

# Retrieves the list of folders with a directory
def getFolderNamesInDirectory(pathToDir, prefix = ""):
    if not isinstance(pathToDir, pathlib.PurePath):
        pathToDir = pathlib.Path(pathToDir)

    return sorted([fld.name for fld in pathToDir.iterdir() if fld.is_dir() and not fld.name.lower().startswith(prefix)])

## 2. Prepare the dataset

The folder structure created through Milestone 1 should looks like this:
```
dataset/train/artCategory_1/file_01.jpg
dataset/train/artCategory_1/file_03.jpg
dataset/train/artCategory_1/file_06.jpg
...
dataset/test/artCategory_1/file_02.jpg
...
dataset/valid/artCategory_1/file_04.jpg
```

The root folder for training is `dataset/train` and the classes are the names of art types.
Likewise, `dataset/valid` and `dataset/test` for validation and testing respectively.

> **Note:**
>   - If you have downloaded images through Milestone 1, the expected folder structure above should be already present
>   - If you have used the set of images (image-sets.zip) provided, please note these images have the following format: 432*288

### Sets the folders
* Set the location for `train`, `test` and `valid` folders
* Count the number of Art category and list them using the function above

In [ ]:
# DEFINE DATA DIRECTORIES & LOCATION OF IMAGE-SETS ARCHIVE
# --------------------------------------------------------

# sets the root folder for image sets
pathToDataset = pathlib.Path.cwd().joinpath('..', 'dataset')
pathToTrain = pathToDataset.joinpath('train')
pathToTest = pathToDataset.joinpath('test')
pathToValid = pathToDataset.joinpath('valid')

# list the folders required under 'dataset' folder (using a list to reduce the lines of code)
artCategories = getFolderNamesInDirectory(pathToTrain, ".")  #collects the list of folders
print("Total no. of categories = ", len(artCategories))  #displays the number of classes (= Art categories)
print("Categories: ", artCategories)  #displays the list of classes

### Transformations

Let's assume that our model expects a `224`-dim square image as input. Resizing will therefore be required for each art image to fit this model. These transformations applies on `train`, `test` and `valid`. Use PyTorch's [ImageFolder](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder) class, which makes it very easy to load data from a directory. 

***A. Data-Augmentation***  
Using data augmentation on training images:

1. Randomly rotating the training by 30 degrees.
2. Randomly cropping and resizing it to 224-dim square image.  
    !!! Please use only one function that crops the given image to random size and aspect ratio.
3. Randomly flipping it horizontally.

> **Note:** 
Data augmentation isn't applied on validation and testing set. These images are resized to 256 pixels and then cropped from center to make it 224-dim square images.  

***B. Normalization:***  
The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]. Same normalization was used by these pre-trained models for training.

---
># TO DO (1):
>## Implement the transformations in the next code paragraph based on the explanation above 


In [ ]:
# Define the data-augmentation transforms including normalisations
train_transforms = transforms.Compose([...,
                                       transforms.ToTensor(),
                                       ...])
                 
test_transforms  = transforms.Compose([...,
                                       transforms.ToTensor(),
                                       ...])
                 
valid_transforms = transforms.Compose([...,
                                       transforms.ToTensor(),
                                       ...])

# load and apply above transforms on dataset using ImageFolder
train_dataset = ...
test_dataset  = ...
valid_dataset = ...

# Print out data stats
print('Training  images: ', len(train_dataset))
print('Testing   images: ', len(test_dataset))
print('Validation images:', len(valid_dataset))

### DataLoaders
A [data loader](https://pytorch.org/docs/stable/data.html) is an iterable over a dataset. The parameters are:
* `batch`:  number of samples per batch to be loaded
* `shuffle`: set to True, data are reshuffled at every epoch.
* `num_workers`: nbr of subprocesses to use for data loading

---
># TO DO (2):
>## Implement the dataloaders in the next code paragraph based on the explanation above.
>* Think on which dataset shuffle should only be applied


In [ ]:
# Define dataloader parameters
batch_size = 16  # You might want to increase the size to 32. This might raise an exception 

num_workers=0

# Prepare data loaders
train_dataloader = ...
test_dataloader  = ...
valid_dataloader = ...

# Print the batches stats
print('Number of  training  batches:', len(train_dataloader))
print('Number of  testing   batches:', len(test_dataloader))
print('Number of validation batches:', len(valid_dataloader))

---
## Optional - Data visualization
> **Note:** the below code is not part of the model training/testing.

---
># TO DO (3 - Optional but recommended):
>## Based on what you have learned above:
> 1. Implement the transformation
> 2. Apply the transformation on the dataset using ImageFolder
> 3. Prepare the dataloader
> * Think on which dataset shuffle should only be applied
> 4. Get a numpy-version of the images


In [ ]:
# Ignore normalization and turn shuffle ON to visualize different art categories together
visual_transforms  = transforms.Compose([...])

# Load and apply above transform on dataset using ImageFolder
# Use test directory images can be used for visualization
visual_dataset  = ...

# Prepare data loaders
visualization_dataloader = ...

# Obtain one batch of testing images
dataiter = iter(visualization_dataloader)
images, labels = dataiter.next()

# Convert images to numpy for display
images = ...

# Plot the images in the batch along with the corresponding labels
plotCols = 4
plotRows = math.ceil(batch_size/plotCols) # SqRoot could be used as well: math.ceil(math.sqrt(batch_size))
fig = plt.figure(figsize=(25, 25))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(plotRows, plotCols, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title(artCategories[labels[idx]])

---
## 3. Modelling

### 3.1. Definition

Several pre-trained models are available and can be used directly. We have selected `resnet50`, `alextnet` and `vgg19`. In this section, we will set the pretrained model to be used, download and load it. Then we will set the different parameters of the features. The model will then be customized to fit specific requirements related to Art classification.

1. Load in a pre-trained model & display its structure  


In [ ]:
# Select the model to use
model_name = 'alexnet' # resnet50, vgg19

# Load the pretrained models from pytorch
model = torch.hub.load('pytorch/vision:v0.6.0', model_name, pretrained=True)

# Print out the model structure
print(model)

2. "Freeze" all the parameters, so the network acts as a fixed feature extractor 

> **Note:** Freezing means that the parameters of features part in the pre-trained model will *not* change during training.


In [ ]:
# Freeze training for all "features" layers
if model_name == 'resnet50': #ResNet50 model
    for param in model.parameters():
        param.requires_grad = False
else:
    for param in model.features.parameters():
        param.requires_grad = False

3. Replace the last layer with a custom linear classifier layer with desirable output nodes

After having the pre-trained feature extractor, modifying final-fully-connected classifier layer

> This layer should produce an appropriate number of outputs for this painting classification task.

You can access any layer in a pre-trained network by name and (sometimes) number.
* For Alexnet/Vgg19: model`.classifier[6]` is the sixth layer in a group of layers named "classifier".
* For ResNet: `resnet50.fc` is the only fully connected Linear layer.

---
># TO DO (4):
># Based on the explanation above 
> 1. Print in/out features
> 2. Set as the last layer of the model a linear layer based on:
>   2.1. the pre-trained model features as input
>   2.2. the number of art categories as output

In [ ]:
# Input features
n_inputs = None
last_layer = None

if model_name == 'resnet50': #ResNet50 model
    ...
else: #AlexNet, Vgg19
    ...

# if GPU is available, move the model to GPU
if train_on_gpu:
    model=model.to(device)

# Print the new model architecture
print(model)

# Check to see that your last layer produces the expected number of outputs
if model_name == 'resnet50':
    print(model.fc.out_features)
else:
    print(model.classifier[6].out_features)


4. Set the model name

In [ ]:
model_filename = 'trained_' + model_name + '.pt'
pathToModel = pathlib.Path.cwd().joinpath('..', 'models', model_filename)
print('File name for saved model: ', pathToModel)

5. Specify [Loss Function](http://pytorch.org/docs/stable/nn.html#loss-functions) and [Optimizer](http://pytorch.org/docs/stable/optim.html)

Below we'll use cross-entropy loss and stochastic gradient descent\. Note that the optimizer accepts as input _only_ the trainable parameters.

> **Note:** AlexNet is made up of 2 parts->  Features (Convolutional and max-pooling layers) and Classifier (Fully-connected layers). In transfer-learning, only the Classifier part is trained.

---
># TO DO (5):
># Based on the explanation above 
> 1. Sets the criterion using categorical cross-entropy combining nn.LogSoftmax() and nn.NLLLoss() in one single class. 
> 2. Sets the appropriate optimizer based on the type of pre-trained models used


In [ ]:
# specify loss function (categorical cross-entropy)
criterion = ...

# specifying optimizer and learning rate
if model_name == 'resnet50':
    optimizer = optim.Adam(..., lr=0.001)
else:
    optimizer = optim.Adam(..., lr=0.001)

### 3.2. Model Training

1. Training Preparation  

    The following variables will be defined:
    * The number of epochs required for the training and validation of the model
    * The minimal value for validation loss
    * Some performance variables such as:
        * the losses for training and validation
        * the model accuracy

In [ ]:
# Some images in dataset were truncated (maybe corrupted)
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Set number of epochs to train the model
n_epochs = 40

# Initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

# Initialise performances
train_losses, valid_losses, accuracies=[],[],[]
training_loss = 0.0
validation_loss = 0.0
accuracy = 0.0


2. Train & Validation the model

    **Training**  
    The training will be performed by going through every epoch.  

    * Collect a set of labelled images and for each image:
        * execute the `forward` method
        * calculate the loss using the criterion and the gradient of the loss
        * apply a single step optimization
        * refresh the training loss

    **Validate**  
    The validation will be performed by going through every epoch - same as the training.

    * Place the model into validation model
        * execute the `forward` method
        * calculate the validation loss and the accuracy
        * compare your predictions against the true labels
        * incrementing values of 'accuracy' with equals
    * Save the model if validation loss has decreased

    Lastly, for each epoch, print the performance variables.

---
># TO DO (6):
># Based on the explanation above 
> 1. Enumerate through a dataloader - think which data_loader should be used here
> 2. Make sure the previous gradients of the optimized variables are cleared
> 3. Calculate the batch loss based on the outputs & labels (in train & validation)
> 4. Calculate the accuracy. The objective here is to return a new tensor based on output values. The tensor will turn outputs into probabilities using exponential function

In [ ]:
a = time.time()  #Start-time for training

for epoch in range(1, n_epochs+1):
    c = time.time()  #Start-time for epoch

    ###############
    # TRAIN MODEL #
    ###############
    
    # Getting one batch of images and their corresponding true labels
    for batch_i, (images, labels) in enumerate(...):

        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images = images.to(device)
            labels = labels.to(device)
            
        # clear the previous/buffer gradients of all optimized variables
        ...
        
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model.forward(images)
        
        # calculate the batch loss
        loss = ...
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        # update training loss 
        training_loss += loss.item()   
        
    ##################
    # VALIDATE MODEL #
    ##################      
    
    model.eval() #model is put to evalution mode i.e. dropout is switched off

    with torch.no_grad():  #Turning off calculation of gradients (not required for validaiton)  {saves time}
        for images, labels in valid_dataloader:   #Getting one batch of validation images
            
            if train_on_gpu:   #moving data to GPU if available
                images = images.to(device)
                labels = labels.to(device)

            outputs = model.forward(images)

            # calculate the batch loss
            batch_loss = ...
            validation_loss += batch_loss.item()
            
            # Calculating accuracy
            ps = ...

            #getting top one probablilty and its corresponding class for batch of images
            top_p, top_class = ps.topk(1, dim=1) 

            #Comparing our predictions to true labels
            equals = top_class == labels.view(*top_class.shape)   #equals is a list of values
            #incrementing values of 'accuracy' with equals
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()   #taking average of equals will give number of true-predictions
                                                #equals if of ByteTensor (boolean), changing it to FloatTensor for taking mean...
    
    
    train_losses.append(training_loss/len(train_dataloader))    
    valid_losses.append(validation_loss/len(valid_dataloader))
    accuracies.append(((accuracy/len(valid_dataloader))*100.0))
    d = time.time() #end-time for epoch
    
    print(f"Epoch {epoch} "
          f"Time: {int((d-c)/60)} min {int(d-c)%60} sec "
          f"Train loss: {training_loss/len(train_dataloader):.3f}.. "
          f"Validation loss: {validation_loss/len(valid_dataloader):.3f}.. "
          f"Validation accuracy: {((accuracy/len(valid_dataloader))*100.0):.3f}% "
          )
    
    training_loss = 0.0
    
    # save model if validation loss has decreased
    if ( validation_loss/len(valid_dataloader) <= valid_loss_min):
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min , validation_loss/len(valid_dataloader)))
        torch.save(model.state_dict(), pathToModel) #Saving model 
        valid_loss_min = validation_loss/len(valid_dataloader)   #Minimum validation loss updated
    
    
    #After validation, model is put to training mode i.e. dropout is again switched on
    model.train()
          
    ################
    # END OF EPOCH #
    ################
            
b = time.time()  #end-time for training
print('\n\n\tTotal training time: ' , int((b-a)/(60*60)), "hour(s) " , int(((b-a)%(60*60))/60),"minute(s) ", int(((b-a)%(60*60))%60) , "second(s)")

3. Load the model  
    The model saved has the lowest validation loss. It is that model that should be loaded.

In [ ]:
model.load_state_dict(torch.load(pathToModel))

### 3.3. Testing

1. Testing Preparation  

    The following variables will be defined:
    * The number of epochs required for the training and validation of the model
    * The minimal value for validation loss
    * Some performance variables such as:
        * the losses for training and validation
        * the model accuracy

In [ ]:
test_loss = 0.0
counter = 0

class_correct = list(0. for i in range(len(artCategories)))
class_total = list(0. for i in range(len(artCategories)))
classes_accuracies=[]

# evaluation mode (switching off dropout)
model.eval()

2. Testing the model  

    * Collect a set of labelled images and for each image:
        * send each image to the model and collect the output
        * based on the output and the label, evaluate the loss
        * convert the output probabilities to a predicted class
        * for each art category, calculate the test accuracy

---
># TO DO (7):
># Based on the explanation above 
> 1. Iterate through a dataloader - think which data_loader should be used here
> 2. Calculate the batch loss based on the outputs & labels (in train & validation)

In [ ]:
y_true = []
y_pred = []

# iterate over test data - get one batch of data from testloader
for images, labels in ...:
    
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        images = images.to(device)
        labels = labels.to(device)
    
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(images)
    
    # calculate the batch loss
    loss = ...
    
    # update  test loss 
    test_loss += loss.item()*images.size(0)
    
    # convert output probabilities to predicted class
    ps, pred = torch.max(output, 1)    
    
    # compare model's predictions to true labels
    for i in range(len(images)):
        y_true.append( artCategories[labels[i]] )
        y_pred.append( artCategories[pred[i]] )

        class_total[labels[i]] += 1
        if pred[i] == labels[i]:
            class_correct[pred[i]] += 1
    counter += 1
    

#calculate avg test loss
test_loss = test_loss/len(test_dataloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(artCategories)):
    classes_accuracies.append(100 * class_correct[i] / class_total[i])
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            artCategories[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (artCategories[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % 
                          (100. * np.sum(class_correct) / np.sum(class_total),np.sum(class_correct), np.sum(class_total)))

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

array = confusion_matrix(y_true, y_pred)

df_cm = pd.DataFrame(array, index = [i for i in artCategories],
                  columns = [i for i in artCategories])
plt.figure(figsize = (10,7))
mx = sn.heatmap(df_cm/np.sum(df_cm), annot=True, fmt='.2%', cmap='Blues')
del df_cm

---
<span style="color:red">

# IMPORTANT NOTE

You might find the accuracy of the models you have generated disappointing. This is due to the fact that these pre-trained models have been trained on photographs as opposed to paintings. In other words, these pre-trained models tend to be biased with regards to the classification of the paintings.

In the next milestone, you will learn how to build your own model and address this type of issue.
</span>
---


## 4. Saving the best model

Additional performance will be saved along with `Training Loss`, `Validations Loss`, `Accuracy`, `Class Accuracy`

In [ ]:
checkpoint = {'training_losses': train_losses,
              'valid_losses': valid_losses,
              'accuracies': accuracies,
              'classes_accuracies':classes_accuracies,
              'state_dict': model.state_dict()}

torch.save(checkpoint, pathToModel)

---
## Optional - Data visualization
> **Note:** the below code is not part of the model training/testing.

In [ ]:
# obtain one batch of test images
dataiter = iter(visualization_dataloader)
images, labels = dataiter.next()
images.numpy()

# move model inputs to cuda, if GPU available
if train_on_gpu:    
    images = images.to(device)
    
# get sample outputs
output = model(images)

#move images to CPU for plotting
images = images.cpu()

# convert output probabilities to predicted class
output_ps, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(22, 22))
for idx in np.arange(8):
    ax = fig.add_subplot(5, 10/2, idx+1, xticks=[], yticks=[])
    plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    ax.set_title("{} ({})".format(artCategories[preds[idx]], artCategories[labels[idx]]),
                 color=("green" if preds[idx]==labels[idx].item() else "red"))
